In [2]:
from sklearn.linear_model import LogisticRegression
clf = LogisticRegression(C=1e10, solver='lbfgs') 

In [22]:
from torch import nn
import torch
import torch.optim as optim

In [64]:
class LogisticRegression(nn.Module):
    def __init__(self, input_dim, output_dim):
        super(LogisticRegression, self).__init__()
        self.linear = nn.Linear(input_dim, output_dim)
        self.sigmoid = nn.Sigmoid()


    def forward(self, x):
        x = self.linear(x)
        x = self.sigmoid(x)
        return x

In [65]:
clf_new = LogisticRegression(2,1)

In [31]:
raw_probs = [0.61051559, 0.00047493709, 0.99639291, 0.00021221573, 0.99599433, 0.0014127002, 0.0028262993]
labels = [1,0,1,0,1,0,0]

import numpy as np
# turn into two-column array, with the i-th column be the probability of the i-th class
raw_probs = np.array(raw_probs) 
raw_probs = np.vstack((raw_probs, 1-raw_probs)).T

In [10]:
clf.fit(raw_probs, labels)
clf.predict(raw_probs)

array([1, 0, 1, 0, 1, 0, 0])

In [70]:
LEARNING_RATE = 0.05
EPOCHS = 1000
loss_function = nn.BCELoss()
optimizer = optim.SGD(clf_new.parameters(), lr=LEARNING_RATE)

In [54]:
raw_probs_tensor = [0.61051559, 0.00047493709, 0.99639291, 0.00021221573, 0.99599433, 0.0014127002, 0.0028262993]

raw_probs_tensor = torch.tensor([[raw_prob, 1-raw_prob] for raw_prob in raw_probs_tensor])
label_tensor = torch.tensor([[float(label)] for label in labels])
print(raw_probs_tensor)
print(raw_probs_tensor.size())
print(label_tensor)
print(label_tensor.size())

tensor([[6.1052e-01, 3.8948e-01],
        [4.7494e-04, 9.9953e-01],
        [9.9639e-01, 3.6071e-03],
        [2.1222e-04, 9.9979e-01],
        [9.9599e-01, 4.0057e-03],
        [1.4127e-03, 9.9859e-01],
        [2.8263e-03, 9.9717e-01]])
torch.Size([7, 2])
tensor([[1.],
        [0.],
        [1.],
        [0.],
        [1.],
        [0.],
        [0.]])
torch.Size([7, 1])


In [71]:
train_losses = []
test_losses  = []
train_accs = []
test_accs  = []

# print(torch.t(torch.tensor([labels])))
for epoch in range(EPOCHS):
    # Forward propagation (predicting train data) #a
    train_preds = clf_new(raw_probs_tensor)
    train_loss  = loss_function(train_preds, label_tensor)
    # print(train_loss)
    
    # Predicting test data #b
    with torch.no_grad():
        test_preds = clf_new(raw_probs_tensor)
        test_loss  = loss_function(test_preds, label_tensor)
        
    # Backward propagation #d
    optimizer.zero_grad()
    train_loss.backward()

    # Gradient descent step #e
    optimizer.step()
    
    # Store training history #f
    train_losses.append(train_loss.item())
    test_losses.append(test_loss.item())

    if epoch%200==0:
        print(f'Epoch: {epoch} \t|' \
            f' Train loss: {np.round(train_loss.item(),3)} \t|' \
            f' Test loss: {np.round(test_loss.item(),3)} \t|')

Epoch: 0 	| Train loss: 0.269 	| Test loss: 0.269 	|
Epoch: 200 	| Train loss: 0.168 	| Test loss: 0.168 	|
Epoch: 400 	| Train loss: 0.124 	| Test loss: 0.124 	|
Epoch: 600 	| Train loss: 0.099 	| Test loss: 0.099 	|
Epoch: 800 	| Train loss: 0.083 	| Test loss: 0.083 	|


In [72]:
clf_new(raw_probs_tensor)

tensor([[0.7855],
        [0.0538],
        [0.9808],
        [0.0538],
        [0.9807],
        [0.0542],
        [0.0547]], grad_fn=<SigmoidBackward0>)